In [1]:
import torch
from ultralytics import YOLO
import supervision as sv
from inference import get_model
from sports.configs.soccer import SoccerPitchConfiguration
from utils import infer_video_and_save_pitch_images, process_offside_detection
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup

**IMPORTANT**

The default model for player/ref/ball detection is our one. If you want to use a pretrained one, select TRUE. Notice that selecting to TRUE will slow down the inference, since the pretrained model is larger.

In [2]:
use_larger_model = False
if use_larger_model:
    PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"
    PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID)
else:
    PLAYER_DETECTION_MODEL = YOLO("best.pt").to(device)

In [ ]:
FIELD_DETECTION_MODEL_ID = "football-field-detection-f07vi/15"
FIELD_DETECTION_MODEL = get_model(model_id=FIELD_DETECTION_MODEL_ID)
PITCH_CONFIGURATION = SoccerPitchConfiguration()
INPUT_VIDEO_PATH = "video.mp4"

# Player Detection

The function saves the video and a folder with the pitch and voronoi projections. Notice that the inference may take up to 1 hour for long videos using the larger model.

In [ ]:
infer_video_and_save_pitch_images(
    player_model=PLAYER_DETECTION_MODEL, field_model=FIELD_DETECTION_MODEL,
    input_video_path=INPUT_VIDEO_PATH, pitch_configuration=PITCH_CONFIGURATION, output_base_folder="inference",
    is_yolov8=use_larger_model,
)

# Offside

It should take a few seconds.

In [ ]:
pitch, _ = process_offside_detection("frame.jpg", PLAYER_DETECTION_MODEL, FIELD_DETECTION_MODEL,
                                     0, 1, PITCH_CONFIGURATION, is_yolov8=use_larger_model)

In [ ]:
sv.plot_image(pitch)